# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\ajvil\Downloads\hw5-super-awesome-team-name-1`
   Installed JpegTurbo_jll ──────── v3.0.4+0
   Installed GR_jll ─────────────── v0.73.8+0
   Installed LERC_jll ───────────── v4.0.0+0
   Installed LoggingExtras ──────── v1.1.0
   Installed OffsetArrays ───────── v1.14.1
   Installed MutableArithmetics ─── v1.5.2
   Installed PlotUtils ──────────── v1.4.3
   Installed NetworkLayout ──────── v0.4.7
   Installed StaticArrays ───────── v1.9.8
   Installed Cairo_jll ──────────── v1.18.2+1
   Installed HTTP ───────────────── v1.10.10
   Installed Libgpg_error_jll ───── v1.50.0+0
   Installed DataFrames ─────────── v1.7.0
   Installed HiGHS_jll ──────────── v1.8.1+0
   Installed Bzip2_jll ──────────── v1.0.8+2
   Installed FFMPEG ─────────────── v0.4.2
   Installed SentinelArrays ─────── v1.4.7
   Installed CodecBzip2 ─────────── v0.8.4
   Installed XZ_jll ─────────────── v5.6.3+0
   Installed PlotThemes ─────────── v3.3.0
   Installed LZO_jll ────────────── v

In [1]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

$\textbf{Decision Variables}$
- $W_{ij}$: Waste transported from city $i$ to disposal $j$ (Mg/day)
- $R_{kj}$: Residual waste transported from disposal $k$ to disposal $j$ (Mg/day)
- $Y_j$: Operational status (on/off) of disposal $j$ (binary)

$\textbf{Objective Function}$

The objective function will be to minimize overall cost, which consist of transportation and disposal costs

$min(Z) = \sum Transportation + \sum Disposal$

$min_{W_{ij}Y_{j}} = \sum_{i}\sum_{j} a_{ij}l_{ij}W_{ij} + \sum_{j}[c_j + \sum_{i}b_jW_{ij}]$

where
- $a_{ij}$ is the cost of transporting waste from source $i$ to disposal $j$ (\$/Mg-km)
- $l_{ij}$ is the distance between source $i$ and disposal $j$ (km)
- $c_j$ is the fixed costs of operating disposal $j$ (\$/day)
- $b_j$ is the variable cost (i.e. tipping cost) of disposing waste at disposal $j$ (\$/Mg) 

However, this holds only if all disposal facilities are operating at the same time. Thus, we need an indicator variable $Y_j$ to designate the option not to operate disposal $j$:

$ Y_j = 
\begin{cases} 
      0 & \text{if} & \sum_{i\in I}W_{ij} = 0 \\
      1 & \text{if} & \sum_{i\in I}W_{ij} > 0
\end{cases}
$

Hence, our objective function will now be:

$min_{W_{ij}Y_{j}} = \sum_{i}\sum_{j} a_{ij}l_{ij}W_{ij} + \sum_{j}[c_jY_j + \sum_{i}b_jW_{ij}]$

Breaking this downs and using specific values:
1. Transportation Costs 

$\sum^3_{i=1}\sum^3_{j=1} 1.5 * l_{ij} * W_{ij}$

$= 1.5[5W_{1,LF} + 30W_{1,MRF} + 15W_{1,WTE} + 15W_{2,LF} + 25W_{2,MRF} + 10W_{2,WTE} + 13W_{3,LF} + 45W_{3,MRF} + 20W_{3,WTE} + 32R_{MRF,LF} + 15R_{MRF,WTE} + 18R_{WTE,LF}]$

2. Disposal Costs

$\sum_{j=1}[c_jY_j + \sum^3_{i=1}b_jW_{ij}]$

where:
- LF: Fixed = \$2000/day, Tipping = \$50/Mg.
- MRF: Fixed = \$1500/day, Tipping = \$7/Mg + \$40/Mg recycled.
- WTE: Fixed = \$2500/day, Tipping = \$60/Mg.

$\textbf{Constraints}$
1. Waste Mass Balance

$\sum_{j}W_{ij} = S_i$

- where $S_i$ is the total waste disposed by city $i$

For each city:
- City 1: $W_{1,LF} + W_{1,MRF} + W_{1,WTE} = 100$
- City 2: $W_{2,LF} + W_{2,MRF} + W_{2,WTE} = 90$
- City 3: $W_{3,LF} + W_{3,MRF} + W_{3,WTE} = 120$

2. Facility capacity limits

 $\sum_{i}W_{ij} + \sum_{k}R_{kj} = K_j$

- where $K_j$ is the capacity limit at each disposal site $j$

3. Recycling and Residual Waste

- $R_{MRF,LF} + R_{MRF,WTE} = (1-.4) * W_{i,MRF}$
- $R_{WTE, LF} = .16 * W_{i,WTE} + .14 * R_{MRF,WTE}$

4. Operational Status

$ Y_j = 
\begin{cases} 
      0 & \text{if} & \sum_{i\in I}W_{ij} = 0 \\
      1 & \text{if} & \sum_{i\in I}W_{ij} > 0
\end{cases}
$

In [ ]:
using JuMP, HiGHS

# Data
cities = 1:3
facilities = 1:3  # 1 = LF, 2 = MRF, 3 = WTE
waste_generated = [100, 90, 120]  # Mg/day
fixed_costs = [2000, 1500, 2500]  # $/day for LF, MRF, WTE
tipping_costs = [50, 7, 60]  # $/Mg for LF, MRF, WTE
recycling_cost = 40  # $/Mg recycled at MRF
transport_cost_per_km = 1.5  # $/Mg-km
capacities = [200, 350, 210]  # Mg/day for LF, MRF, WTE
distances = [
    [5, 30, 15],   # City 1 to LF, MRF, WTE
    [15, 25, 10],  # City 2 to LF, MRF, WTE
    [13, 45, 20]   # City 3 to LF, MRF, WTE
]

# Recycling and residual rates
recycling_rate = 0.4
ash_fraction_non_recycled = 0.16
ash_fraction_recycled = 0.14

# Model
model = Model(HiGHS.Optimizer)

# Variables
@variable(model, W[cities, facilities] >= 0)  # Waste transported (Mg/day)
@variable(model, R[facilities, facilities] >= 0)  # Residuals transported (Mg/day)
@variable(model, Y[facilities], Bin)  # Operational status of facilities

# Objective function: Minimize total cost
@objective(
    model,
    Min,
    # Fixed costs
    sum(fixed_costs[j] * Y[j] for j in facilities) +
    # Transportation costs: city to facility 
    sum(transport_cost_per_km * distances[i][j] * W[i, j] for i in cities, j in facilities) +
    # Transportation costs: facility to facility 
    (32*R[2,1] + 15*R[2,3] + 18*R[3,1]) + 
    # Tipping costs: city to facility 
    sum(tipping_costs[j] * W[i, j] for i in cities, j in facilities) +
    # Tipping costs: facility to facility 
    (50*R[2,1] + 60*R[2,3] + 50*R[3,1]) + 
    # Recycling cost at MRF
    recycling_cost * recycling_rate * sum(W[i, 2] for i in cities)  
)

# Constraints
# Waste mass balance for each city
for i in cities
    @constraint(model, sum(W[i, j] for j in facilities) == waste_generated[i])
end

# Facility capacity constraints
for j in facilities
    @constraint(model, sum(W[i, j] for i in cities) + sum(R[k, j] for k in facilities if k != j) <= capacities[j])
end

# Recycling and residual waste constraints
@constraint(model, R[2, 1] + R[2,3] == (1 - recycling_rate) * sum(W[i, 2] for i in cities))  
@constraint(model, R[3, 1] == (ash_fraction_non_recycled * sum(W[i, 3] for i in cities) + (ash_fraction_recycled * R[2,3]))) 

@constraint(model, R[1,1] + R[1,2] + R[1,3] + R[2,2] + R[3,2] + R[3,3] == 0)

# Indicator Variable constraint 
@constraint(model, commit1, 1000Y[1] >= sum(W[i, 1] for i in cities) + R[2,1] + R[3,1]) # LF
@constraint(model, commit2, 1000Y[2] >= sum(W[i, 2] for i in cities))  # MRF
@constraint(model, commit3, 1000Y[3] >= sum(W[i, 3] for i in cities) + R[2,3]) # WTE

# Solve the model
optimize!(model)

# Results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal cost: ", objective_value(model))
    println("Waste transported:")
    for i in cities, j in facilities
        println("City $i to Facility $j: ", value(W[i, j]), " Mg/day")
    end
    println("Residual waste transported:")
    for k in facilities, j in facilities
        println("Facility $k to Facility $j: ", value(R[k, j]), " Mg/day")
    end
    println("Facility operational status:")
    for j in facilities
        println("Facility $j: ", value(Y[j]) == 1 ? "Operational" : "Not Operational")
    end
else
    println("Optimization did not converge.")
end

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-01, 1e+03]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [9e+01, 4e+02]
Presolving model
10 rows, 15 cols, 43 nonzeros  0s
8 rows, 13 cols, 35 nonzeros  0s

Solving MIP model with:
   8 rows
   13 cols (2 binary, 0 integer, 0 implied int., 11 continuous)
   35 nonzeros
MIP-Timing:       0.011 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

      

Based on the solution, Facility 2 (MRF) is not operational. Facilities 1 (LF) and 3 (WTE) are operational, with waste allocated as follows:
- City 1 sends its waste entirely to Facility 1 (LF).
- City 2 sends its waste entirely to Facility 3 (WTE).
- City 3 splits its waste between Facilities 1 (LF) and 3 (WTE).

Several reasons the MRF was not used:
- MRF has a low tipping cost (7/Mg) but incurs an additional recycling cost (40/Mg). This cost may outweigh the benefits of recycling in this specific setup.
- MRF is farther from the cities compared to the other facilities. Transportation costs (\$1.5/Mg-km) add significantly to the total cost of waste sent to this facility, making it less favorable.
- The total waste produced by the cities (310 Mg/day) fits within the combined capacities of LF (200 Mg/day) and WTE (210 Mg/day). Hence, MRF is not required to handle overflow.

With these compounded reasons, it makes sense why the MRF was not used. The diagram for waste allocation is shown below. 

## References

List any external references consulted, including classmates.